# 在 PyTorch 中实现字符级 LSTM

在此 notebook 中，我将使用 PyTorch 构建一个字符级 LSTM。该网络将根据一段文本逐个字符地接受训练，并逐个字符地生成新的文本。我将用《安娜·卡列尼娜》训练网络。**此模型将能够根据这本小说里的文字生成新的文本。**

此网络参考了 Andrej Karpathy 的 [RNN 帖子](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)和 [Torch 中的实现](https://github.com/karpathy/char-rnn)。下面是这个字符级 RNN 的一般结构。

<img src="assets/charseq.jpeg" width="500">

首先加载资源，以便加载数据和创建模型。

In [16]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## 加载数据

然后，我们将加载《安娜·卡列尼娜》文本文件并将其转换为整数，这样才能使用于网络中。

In [17]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

In [18]:
len(text)

1985223

检查前 100 个字符，看看是否一切顺利。根据[美国图书评论](http://americanbookreview.org/100bestlines.asp)，这本小说的开篇第一句是有史以来排名第六的图书名言佳句。

In [19]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### 标记化

在下面的单元格中，我创建了几个在字符和整数之间进行转换的**字典**。将字符变成整数便于我们将它们当做输入传入网络中。

In [20]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

可以看到上面的字符变成了整数。

In [21]:
encoded[:100]

array([79, 51, 26, 38, 27, 41, 67, 22, 74, 78, 78, 78, 71, 26, 38, 38, 54,
       22, 29, 26, 34, 55, 36, 55, 41,  6, 22, 26, 67, 41, 22, 26, 36, 36,
       22, 26, 36, 55, 58, 41, 42, 22, 41, 65, 41, 67, 54, 22, 21, 15, 51,
       26, 38, 38, 54, 22, 29, 26, 34, 55, 36, 54, 22, 55,  6, 22, 21, 15,
       51, 26, 38, 38, 54, 22, 55, 15, 22, 55, 27,  6, 22, 24, 57, 15, 78,
       57, 26, 54, 39, 78, 78, 31, 65, 41, 67, 54, 27, 51, 55, 15])

## 预处理数据

从上面的 char-RNN 示意图中可以看出，LSTM 层级要求输入是**独热编码**，意思是（通过创建的字典）将每个字符转换为一个整数，*然后*转换为列向量，只有相应的整数索引值将为 1，向量的其他部分全为 0。因为我们要独热编码数据，所以创建一个函数来执行此操作。

In [22]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [23]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[ 0.  0.  0.  1.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  1.  0.  0.]
  [ 0.  1.  0.  0.  0.  0.  0.  0.]]]


## 创建训练迷你批次


要使用此数据训练模型，我们还需要创建迷你批次。我们希望批次是多个序列，由一定数量的序列步组成。下面是一个简单的批次示例：

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

在此示例中，我们将获取编码字符（作为 `arr` 参数传入），并根据 `batch_size` 将它们拆分为多个序列。每个序列长为 `seq_length`。

### 创建批次

**1.首先我们需要丢弃一些文本，从而创建完整的迷你批次。**

每个批次包含 $N \times M$ 个字符，其中 $N$ 是批次大小（一个批次中的序列数量），$M$ 是 seq_length，即一个序列中的时间步数。要获取可以根据数组 `arr` 创建的批次总数 $K$，将 `arr` 的长度除以每个批次的字符数。获取批次数量后，就能算出可以从 `arr` 中保留的字符总数：$N * M * K$。

**2.之后，我们需要将 `arr` 拆分为 $N$ 批。** 

我们可以使用 `arr.reshape(size)`，其中 `size` 是一个元组，包含变形数组的维度大小。我们希望一个批次里有 $N$ 个序列，所以将其设为第一个维度的大小。对于第二个维度，你可以使用 `-1` 作为大小占位符，它将使用合适的数据填充数组。之后就获得一个 $N \times (M * K)$ 数组。

**3.获得此数组后，我们可以遍历它以获取迷你批次。**

每个批次是应用在 $N \times (M * K)$ 数组之上的 $N \times M$ 窗口。对于每个后续批次，窗口将移动 `seq_length` 个位置。我们还需要创建输入和目标数组。目标是偏移一个字符的输入。对于此窗口，我将通过 `range` 采取一定的步数，步数为 `n_steps`，从 $0$ 到 `arr.shape[1]`，即每个序列中的标记总数。这样的话，从 `range` 获取的整数始终指向一个批次的开头，每个窗口的宽为 `seq_length`。

> **TODO：**在下面的函数中编写创建批次的代码。此 notebook 中的练习_不太容易_。除了此 notebook 之外，我还提供了 solution notebook。如果你遇到问题，请参阅 solution notebook。最重要的一点是，不要只复制粘贴代码，而是**自己试着编写代码。**

In [24]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### 测试实现代码

现在我将创建一些数据集，看看批处理数据会发生什么。我将批次大小设为 8，序列步数设为 50。

In [25]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)
print(x.shape, y.shape)

(8, 50) (8, 50)


In [26]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[79 51 26 38 27 41 67 22 74 78]
 [ 6 24 15 22 27 51 26 27 22 26]
 [41 15  2 22 24 67 22 26 22 29]
 [ 6 22 27 51 41 22 75 51 55 41]
 [22  6 26 57 22 51 41 67 22 27]
 [75 21  6  6 55 24 15 22 26 15]
 [22 68 15 15 26 22 51 26  2 22]
 [80 20 36 24 15  6 58 54 39 22]]

y
 [[51 26 38 27 41 67 22 74 78 78]
 [24 15 22 27 51 26 27 22 26 27]
 [15  2 22 24 67 22 26 22 29 24]
 [22 27 51 41 22 75 51 55 41 29]
 [ 6 26 57 22 51 41 67 22 27 41]
 [21  6  6 55 24 15 22 26 15  2]
 [68 15 15 26 22 51 26  2 22  6]
 [20 36 24 15  6 58 54 39 22 53]]


如果你正确地实现了 `get_batches`，上述输出应该如下所示：

```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 当然实际数字可能有所不同。检查 `y` 的数据是否偏移了一个位置。

---
## 在 PyTorch 中定义网络

你将在下面定义网络。

<img src="assets/charRNN.png" width=500px>

接下来，你将使用 PyTorch 定义网络结构。首先定义层级和要执行的运算。然后定义前向传递方法。我们还提供了预测字符的方法。

### 模型结构

在 `__init__` 中，建议的结构如下所示：
* 创建并存储必要的字典（已经替你完成这一步）
* 定义一个接受以下参数的 LSTM 层级：输入大小（字符数）、隐藏层大小 `n_hidden`、层数 `n_layers`、丢弃概率 `drop_prob`，以及 batch_first 布尔值（设为 True，因为我们要批处理）
* 使用 `dropout_prob` 定义丢弃层
* 定义一个接受以下参数的全连接层：输入大小 `n_hidden` 和输出大小（字符数）
* 最后，初始化权重（已经替你完成这一步）

注意，`__init__` 函数已经命名并提供某些参数，我们以 `self.drop_prob = drop_prob` 格式使用并存储这些参数。

---
### LSTM 输入/输出

你可以如下所示地创建基本 [LSTM 层级](https://pytorch.org/docs/stable/nn.html#lstm)

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

其中 `input_size` 是此单元希望序列输入具有的字符数，`n_hidden` 是单元中的隐藏层里包含的单元数量。要添加丢弃层，我们可以添加丢弃参数并指定丢弃概率；丢弃参数会自动向输入或输出添加丢弃层。最后，在 `forward` 函数中使用 `.view` 将 LSTM 单元堆叠为层。然后传入单元列表，它会将一个单元的输出传入下个单元。

还需要将隐藏状态全初始化为 0。代码如下所示：

```python
self.init_hidden()
```

In [27]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [28]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
#         out = out.contiguous().view(-1, self.n_hidden)
        out = out.permute(1, 0, 2).contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## 训练模型

我们将在 train() 函数中设定周期数、学习速率和其他参数。

我们在下面使用 Adam 优化器和交叉熵损失，因为输出是字符类别分数。照常计算损失并执行反向传播步骤。

关于训练的几个细节信息： 
>* 我们在训练循环中将隐藏状态与其历史记录分离开；这次将其设为新的*元组*变量，因为 LSTM 有一个隐藏状态，该隐藏状态是由隐藏状态和单元状态组成的元组。
* 我们使用[`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) 防止梯度爆炸。

In [36]:
from workspace_utils import keep_awake

In [37]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
#     for e in range(epochs):
    for e in keep_awake(range(epochs)):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## 实例化模型

现在可以正式训练网络了。首先创建网络并传入一些超参数。然后定义迷你批次大小，就可以开始训练了。

In [38]:
## TODO: set you model hyperparameters
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


### 设置训练超参数。

In [39]:
from workspace_utils import active_session

with active_session():
    print('import sucess')

import sucess


In [40]:
batch_size = 128
seq_length = 100
n_epochs = 5 # start smaller if you are just testing initial behavior

# train the model
with active_session():
    train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/5... Step: 10... Loss: 3.2460... Val Loss: 3.1804
Epoch: 1/5... Step: 20... Loss: 3.1417... Val Loss: 3.1298
Epoch: 1/5... Step: 30... Loss: 3.1371... Val Loss: 3.1228
Epoch: 1/5... Step: 40... Loss: 3.1119... Val Loss: 3.1210
Epoch: 1/5... Step: 50... Loss: 3.1438... Val Loss: 3.1192
Epoch: 1/5... Step: 60... Loss: 3.1210... Val Loss: 3.1183
Epoch: 1/5... Step: 70... Loss: 3.1104... Val Loss: 3.1187
Epoch: 1/5... Step: 80... Loss: 3.1297... Val Loss: 3.1189
Epoch: 1/5... Step: 90... Loss: 3.1308... Val Loss: 3.1189
Epoch: 1/5... Step: 100... Loss: 3.1272... Val Loss: 3.1191
Epoch: 1/5... Step: 110... Loss: 3.1280... Val Loss: 3.1186
Epoch: 1/5... Step: 120... Loss: 3.1114... Val Loss: 3.1182
Epoch: 1/5... Step: 130... Loss: 3.1279... Val Loss: 3.1183
Epoch: 2/5... Step: 140... Loss: 3.1285... Val Loss: 3.1185
Epoch: 2/5... Step: 150... Loss: 3.1300... Val Loss: 3.1182
Epoch: 2/5... Step: 160... Loss: 3.1198... Val Loss: 3.1184
Epoch: 2/5... Step: 170... Loss: 3.0968... Val Lo

## 获得最佳模型

要设置超参数以获得最佳效果，你需要观察训练和验证损失。如果训练损失比验证损失低多了，则表明出现过拟合现象。请添加正则化（更多丢弃层）或使用更小的网络。如果训练和验证损失很接近，则说明网络欠拟合，所以你需要增大网络。

## 超参数

下面是网络的超参数。

在模型定义部分：
* `n_hidden` - 隐藏层的单元数量。
* `n_layers` - 隐藏 LSTM 层级的数量。

在此例中，我们将保留默认的丢弃概率和学习速率。

在训练部分：
* `batch_size` - 一次经过网络的序列数量。
* `seq_length` - 用于训练网络的序列中的字符数。通常网络越大越好，使网络能够学习更多的上下文信息。但是训练时间更长。通常 100 个就很合适。
* `lr` - 训练过程中的学习速率

下面是 Andrej Karpathy 提出的一些网络训练的好建议。我直接摘录了他的建议，你也可以点击此链接查看[原文](https://github.com/karpathy/char-rnn#tips-and-tricks)。

> ## 提示和技巧

>### 监控验证损失与训练损失
>如果你是机器学习或神经网络新手，那么获取好的模型需要一些技巧。要记录的最重要指标是训练损失（在训练期间输出）和验证损失（当 RNN 在验证数据上运行时，每隔一段时间输出验证损失，默认情况下是每迭代 1000 次）之间的差别。尤其是：

> - 如果训练损失比验证损失低得多，则表明网络可能**过拟合**了。解决方案是缩小网络，或增加丢弃率。例如，你可以尝试将丢弃率设为 0.5，等等。
> - 如果训练/验证损失差不多大，那么模型**欠拟合**了。请增大模型（可以增加层级数量或每个层级的原始神经元数量）

> ### 估计参数值

> 控制模型的两个最重要的参数是 `n_hidden` 和 `n_layers`。建议 `n_layers` 始终设为 2 或 3。可以根据数据量调整 `n_hidden`。要记录的两个最重要的指标是：

> - 模型中的参数数量。在开始训练时输出该数量。
> - 数据集大小。1MB 文件约为 100 万个字符。

>这两个应该约为相同的数量级。这些参数比较难设定。下面是一些示例：

> - 我有一个 100MB 的数据集，我使用默认的参数设置（当前会输出 15 万个参数）。数据大得多（1 亿远远大于15 万），所以肯定会严重欠拟合。我完全可以增大 `n_hidden`。
> - 我有一个 10MB 的数据集，并且模型有 1000 万个参数。也就是说，我们需要仔细监控验证损失。如果大于训练损失，那么就需要稍微增大丢弃概率，看看能否降低验证损失。

> ### 最佳模型策略

>获得很好模型的最佳策略（如果你有计算时间）是始终尝试更大的模型（大到愿意花费一定的计算时间），然后尝试不同的丢弃值（在 0 到 1 之间）。最终应该使用验证效果最好的模型（写入检查点文件里的损失越低越好）。

>在深度学习领域，最常见的做法是用很多不同的超参数设置运行大量不同的模型，最终选择验证损失最佳的检查点。

>顺便提下，训练集和验证集划分也是参数。验证集中应该有足够的数据，否则验证效果将有噪点，并且不能提供可靠的信息。

## 检查点

训练之后，我们将保存模型，方便以后重新加载模型。我将保存创建相同架构所需的参数、隐藏层超参数和文本字符。

In [41]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## 做出预测

训练好模型后，我们需要从中抽样并预测下个字符。为了抽样，我们传入一个字符并使网络预测下个字符。然后将该字符传入网络中并预测下个字符。不断这么操作，直到生成大量文本。

### 关于 `predict` 函数的注意事项

RNN 的输出来自全连接层，它会输出**下个字符的分数分布**。

> 要获得下个字符，我们应用 softmax 函数，它会提供*概率*分布，我们然后从中抽样并预测下个字符。

### Top-K 抽样

预测来自所有潜在字符的类别概率分布。我们可以抽样文本并仅考虑前 $K$ 个潜在字符，使抽样文本更合理（变量更少）。这样可以避免网络提供完全不合理的字符，并且能够向抽样文本里引入一些噪点和随机性。详细了解 [top-k](https://pytorch.org/docs/stable/torch.html#torch.topk)。

In [42]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### 设定 prime 单词并生成文本 

通常，需要设定 prime 单词来构建隐藏状态。否则，网络将开始随机生成字符。前几个字符一般比较难预测，因为预测上下文信息不足。

In [43]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [44]:
print(sample(net, 1000, prime='Anna', top_k=5))

Annatoo oe tooee ettao   e  eee oaoete e  ooete  eoe o oottoeaeotte attoeoe aeaa e   o   o oe  a   e  aoe toa  eeeteeaetaeo ta tea tta   o oeet ottoatt  t tt   aaet ooo e  te ot eee  eotttaeo taeo tta eetee  aoeoa   e eaeaotoae a  eto e otea t ttee atooe e  eoo       etoe   toeoeeatta  ottoeaatee    otee eat t  toeato  ato  eeto tte  eeaoeoa  aaeeototaet t    o   e e      at aa  eteet   eet eaaea   eettt a oa  toa ao tta te     t  oa eattt  eo teae  tto eeaeeoaeo aetae   o e  ee    et te e  o teoo a eea tte a e t     ooteaooetoo e t oteto teatoteee tee ea o o ee tooa ae   oetae t ee aatoe ooa e ea  t at o ao t et o  o    o tae oo aa  atatoe at e  ooee teete eto oe  e  e a te    oa ae   aaea  a ttte   oaa ta   ata ta ao t o a   ee  ooto t  e etatta e oaoe oo  eaea oo aatt      et etattteeo tete   o ao  taeat oaoe a o  o e oaaoe eetee ae  oe oeo ete ao    o  ee ett  eeeeaeaoeote et o eaetao o     aate etoet tt  oteaa   e    toao  e e teate  ooo atooa e    oe  oeeoe tooeeet t a    o  o  a

## 加载检查点

In [45]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [46]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin saidoettaa   aettteaaeteett oo  o o t tt eeeaot  aoeetet  aoao  t to e  oe  e  eeoaae  ttaaae   et o    oe a a a atat oaoo     t  tt  o eata aa ooo t etaa  e e e   a   ee    e  eto  t et e eta   t  oe aae oo et  eao  e  a e ot teeot   eo ee  t  et    t otet  tt t eaa  t  aa    a a  aete t  oo t  a  oeee  ate    otet   ea a  at oat oetoto t  tooa ooettteae  te  a   a   ea t  ae  ottt  e  o  eeaeatoott e eta e  e  ae ae  et etaetoteoaeeet tetata t eteaoaataeet otot       e  ot ea aeaeao  a e e toa t toa aeee e  ooeea t ae  ot teoee e eoeeott  eaetot oo tt t eoott a oa  oa et e taae   eo   aeooae  eeooao   a aa e ota eeoa  eaet attaat  oeeo  e  ttoa   aeo   taet  eooae eoattaaote t  ee ee  aotataoaettaa ao ttet e  t eo teett o   oa ete e o eo oeeeea  a et eaooot a   a  eaa  aeeete t ato  e  a o e too toeeo  eaoeeaaaaa oooaoeoo     a  e  e  t  e   tea oe  oatea otett et otoee  t  eo a   ee e  o eeaot  etee ae ea  teo ae    otteoee  o aaee o a tt a eea t o   oao t   aaee  ao t ao 

```python

```